In [ ]:
from requests_html import HTMLSession
import time
import pymysql
import random

import mysql.connector
import urllib.request
import   sqlhelper
import   utils
import re
import os



def getTitleImg(url,id):
    path = 'E:\\phpstudy_pro\\WWW\\images\\' +str(id)+'\\' 
   # 将图片保存到D:\\test文件夹中，如果没有test文件夹则创建
    if not os.path.isdir(path):  
        os.makedirs(path)  
    paths = path+'\\'      #保存在test路径下 
    x="imgtitle"+str(id)
    imgurl="http:"+url
    ext=(imgurl.rpartition("."))[2]
    print(ext) 
    print("titleimgurl=",imgurl)
    urllib.request.urlretrieve(imgurl,'{0}{1}.{2}'.format(paths,x,ext))
    titleimgurl="/images/"+str(id)+"/imgtitle"+str(id)+".jpg"  
    print("测试：titleimgurl：---",titleimgurl)
    return  titleimgurl



sql="SELECT * FROM urls WHERE  date2 IS NULL"
queryresult=sqlhelper.select_all(sql,[])

#print(len(queryresult))


for x in queryresult:
  try:
    categoryid=x[0]
    print(categoryid)
    category_id=int(x[0])
    url_t=x[1]
    print(x[1])   #https://list.jd.com/list.html?cat=670,671,674
  #构造地址链接  670%2C671%2C674&page=3&s=61&click=0
 # 

   # url_t="https://list.jd.com/list.html?cat=1620,11158,11165"
    for i in range(1,100,2):
        id=0  #初始值
        try:
         #  url=url_t+"&page="+str(i)+"&s=61&click=0"
           url=url_t+"&page="+str(i)+"&s=11&click=0"
           print(url)
           second=random.randint(20,100)
           print("抓取页面间隔{0}秒".format(second))
           time.sleep(second)
            
           
      #label .begin
           session = HTMLSession()
     # try:
           r = session.get(url)
           url=url_t
        
           list_a=r.html.find('div#J_goodsList')

           path="//li"
#print(list_a[0].html)
           content=list_a[0].html
           path2="//@data-lazy-img"
          # path3="//em/text()"
          # path3="//div[@class='p-name']/a/@title"
           path3="//div/a/em/text()"  
           path4="//i/text()"
           path5="//a/text()"
           path6="//span[@class='J_im_icon']/a/text()"
           path7="//a[@class='curr-shop hd-shopname']/text()"
           path8="//a/img[data-lazy-img]"
           path9="//div[@class='p-img']/a/@href"
           path10="//href"
           id=0
#  content="<html><body style='text-align:center;'>京东产品列表页<table  border='1' style='border-collapse:collapse;margin:0px auto;border:1px solid ;'><tr style='background-color:red;'><th>选择框</th><th>ID</th><th>产品图</th><th>产品名</th><th>价格</th><th>店铺名</th><th>生成时间</th><th>操作</th></tr>"
           valist=[]
         #  time.sleep(random.randint(15,110))
           for li in (r.html.find('div#J_goodsList'))[0].xpath(path):

                   img_url=li.xpath(path2)

                   name=li.xpath(path3)[0]
                   name=name.replace('\r','').replace('\n','').replace('\t','')
                   print(name)
    #   

     #  print(li.xpath(path4)[0])
    #   print(li.xpath(path5)[0])    
                   price=li.xpath(path4)[0]
   #    print(li.xpath(path6))
                   store_name=li.xpath(path7)
                   s_name=""
                   if store_name==[]:
                        s_name=""
                   else:
                        s_name=store_name[0]
            
      # print(s_name)
                   product_url=li.xpath(path9)
                   if "item.jd.com"  in product_url[0]:
       
                       sql="select * from spider where url=%s"   #查询地址是否已经添加到数据库
                       print("product_url[0]:",product_url[0])
                       result=sqlhelper.select_one(sql, product_url[0]) 
                       print("result:",result)
                       if   result == None:
                            # titleimgurl=getTitleImg(img_url[0],id)
                             tup1 = [name,img_url[0],price,s_name,time.strftime('%Y-%m-%d %H:%M:%S',time.localtime()),category_id,product_url[0]]    
                  # valist.append(tup1)
                           
                             sql = "INSERT INTO spider(title, img, price, store,date,categoryid,url) VALUES (%s, %s,%s, %s, %s, %s,%s )"
                            # print(res, "记录写入成功。")
                             res = sqlhelper.insert_one(sql, tup1)
                             print(res, "记录写入成功。")                                
                                
                             sql="select * from spider where img=%s"
                             resultimg=sqlhelper.select_one(sql, img_url[0]) 
                             print("id：",resultimg[0])
                             print("标题图地址：",resultimg[2])
                             id=resultimg[0]
                             newtitleimgurl=getTitleImg(img_url[0],id)
                             

                             print("newtitleimgurl:",newtitleimgurl)

                             sql = "UPDATE spider SET img=%s WHERE id= %s"
                       
                             booleantag=sqlhelper.update_one(sql,[newtitleimgurl,str(id)])
                             print("booleantag=",booleantag)

           #time1=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())
           
          # mycursor.executemany(sql, tuple(valist))
        except:
                delpath = 'f:\\phpstudy_pro\\WWW\\images\\' +str(id)+'\\' 
                remove_dir(delpath,print_result=True)
                time.sleep(random.randint(15,110))
                continue
   # sql = "INSERT INTO spider(title, img, price, store,date,categoryid,url) VALUES (%s, %s,%s, %s, %s, %s,%s )"
    sql = "UPDATE urls SET tag = %s,date2=%s  WHERE id= %s"
    res = sqlhelper.update_one(sql, ["Y",time.strftime('%Y-%m-%d %H:%M:%S',time.localtime()),str(category_id)])
    print(res, "一个产品分类地址记录写完。") 
  except:
                second=random.randint(1500,250)
                print("抓取异常休息{}秒再继续，时间点：{2}".format(second,time1=time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())))
                time.sleep(second)
                continue
        
        

139
https://list.jd.com/list.html?cat=737,14857,14860
https://list.jd.com/list.html?cat=737,14857,14860&page=1&s=11&click=0
抓取页面间隔93秒
复旦申花 豆浆机商用大容量 全自动加热一体磨浆机 大型现磨打浆机器 干湿两用电动煮豆浆 10升（一次可打50杯）
product_url[0]: //item.jd.com/10026655646998.html
result: None
True 记录写入成功。
id： 53226
标题图地址： //img13.360buyimg.com/n7/jfs/t1/167391/31/15607/124661/605d4292Ec3393d44/a4997cea4aa680b4.jpg
jpg
titleimgurl= http://img13.360buyimg.com/n7/jfs/t1/167391/31/15607/124661/605d4292Ec3393d44/a4997cea4aa680b4.jpg
测试：titleimgurl：--- /images/53226/imgtitle53226.jpg
newtitleimgurl: /images/53226/imgtitle53226.jpg
booleantag= True
德玛仕（DEMASHI）现磨豆浆机 破壁机商用家用 榨汁机沙冰机榨汁机多功能免滤全自动大功率料理机 5升杯体多功能款（不加热）
product_url[0]: //item.jd.com/10022992184115.html
result: None
True 记录写入成功。
id： 53227
标题图地址： //img10.360buyimg.com/n7/jfs/t1/170927/38/14859/220617/605ca10cE14ece037/f757350e25e09021.jpg
jpg
titleimgurl= http://img10.360buyimg.com/n7/jfs/t1/170927/38/14859/220617/605ca10cE14ece037/f757350e25e09021.jpg
测试：titleimgurl：--- /ima

测试：titleimgurl：--- /images/53242/imgtitle53242.jpg
newtitleimgurl: /images/53242/imgtitle53242.jpg
booleantag= True
【升级新款】商用豆浆机全自动渣浆分离大容量早餐店用现磨12-40L大型无渣免滤迪巴鲁 12升豪华款(内置滤网+24小时预约)
product_url[0]: //item.jd.com/10025283079960.html
result: None
True 记录写入成功。
id： 53243
标题图地址： //img10.360buyimg.com/n7/jfs/t1/159681/21/12940/178310/604d7c93E33f54b4d/8e64548e8a21232a.jpg
jpg
titleimgurl= http://img10.360buyimg.com/n7/jfs/t1/159681/21/12940/178310/604d7c93E33f54b4d/8e64548e8a21232a.jpg
测试：titleimgurl：--- /images/53243/imgtitle53243.jpg
newtitleimgurl: /images/53243/imgtitle53243.jpg
booleantag= True
巧厨娘磨浆机商用渣浆分离多功能全自动家用小型现磨豆浆机豆腐机100型
product_url[0]: //item.jd.com/67035469513.html
result: None
True 记录写入成功。
id： 53244
标题图地址： //img13.360buyimg.com/n7/jfs/t1/168189/15/16353/125530/606a98f4E155df395/0908ab7b97a02906.jpg
jpg
titleimgurl= http://img13.360buyimg.com/n7/jfs/t1/168189/15/16353/125530/606a98f4E155df395/0908ab7b97a02906.jpg
测试：titleimgurl：--- /images/53244/imgtitle53244.jpg
newtitleimgurl: 

测试：titleimgurl：--- /images/53259/imgtitle53259.jpg
newtitleimgurl: /images/53259/imgtitle53259.jpg
booleantag= True
【早餐店推荐款】豆浆机商用全自动渣浆分离早餐店餐厅食堂用蒸汽煮浆豆腐机一体大型磨浆机迪巴鲁 40型燃气豆浆一体机加强款220V
product_url[0]: //item.jd.com/10029381636322.html
result: None
True 记录写入成功。
id： 53260
标题图地址： //img12.360buyimg.com/n7/jfs/t1/183678/35/482/110097/60823e10Edb0cd2e0/b015a8769d11e78e.jpg
jpg
titleimgurl= http://img12.360buyimg.com/n7/jfs/t1/183678/35/482/110097/60823e10Edb0cd2e0/b015a8769d11e78e.jpg
测试：titleimgurl：--- /images/53260/imgtitle53260.jpg
newtitleimgurl: /images/53260/imgtitle53260.jpg
booleantag= True
【早餐店推荐款】豆浆机商用全自动渣浆分离早餐店餐厅食堂用蒸汽煮浆豆腐机一体大型磨浆机迪巴鲁 50型燃气豆浆一体机升级款220V
product_url[0]: //item.jd.com/10029381636325.html
result: None
True 记录写入成功。
id： 53261
标题图地址： //img10.360buyimg.com/n7/jfs/t1/180858/34/538/110611/60823e8dE5f9914f4/052fb8398c512ae7.jpg
jpg
titleimgurl= http://img10.360buyimg.com/n7/jfs/t1/180858/34/538/110611/60823e8dE5f9914f4/052fb8398c512ae7.jpg
测试：titleimgurl：--- /images/53261/imgtitle5

测试：titleimgurl：--- /images/53276/imgtitle53276.jpg
newtitleimgurl: /images/53276/imgtitle53276.jpg
booleantag= True
喜莱盛商用豆浆机商用大容量不锈钢无渣早餐店食堂幼儿园大型现磨全自动加热打浆机器 干湿两用电动煮豆浆 12升智能保温款
product_url[0]: //item.jd.com/10026223998271.html
result: None
True 记录写入成功。
id： 53277
标题图地址： //img11.360buyimg.com/n7/jfs/t1/174135/7/470/140226/605d60e3E2fee795c/794e89616f0358e4.jpg
jpg
titleimgurl= http://img11.360buyimg.com/n7/jfs/t1/174135/7/470/140226/605d60e3E2fee795c/794e89616f0358e4.jpg
测试：titleimgurl：--- /images/53277/imgtitle53277.jpg
newtitleimgurl: /images/53277/imgtitle53277.jpg
booleantag= True
https://list.jd.com/list.html?cat=737,14857,14860&page=5&s=11&click=0
抓取页面间隔57秒
